# Notebook 05: Interpretability and Directionality Analysis

**Understanding What Drives Willingness to Apply to AI-Hiring Jobs**

This notebook provides:
- Part E: Coefficient analysis with confidence intervals (Logistic Regression)
- Part E: SHAP values with directionality interpretation (Gradient Boosting)
- Comparison of importance methods
- Part F: Fairness and subgroup diagnostics

**Target Definition Reminder:**
- `y=1`: Would apply (response = "Yes, I would")
- `y=0`: Would NOT apply (response = "No, I would not")

**Interpretation Guide:**
- Positive coefficients → INCREASE probability of applying
- Positive SHAP values → INCREASE probability of applying

In [ ]:
# Setup
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

from src.data_loading import load_atp_w119, create_target_variable, get_feature_columns
from src.preprocessing import prepare_modeling_data, create_train_test_split, scale_features
from src.interpretability import (get_logistic_coefficients, bootstrap_logistic_coefficients,
                                   compute_shap_values, shap_summary_table, 
                                   compute_permutation_importance, compare_importance_methods,
                                   interpret_coefficient, interpret_shap_feature)
from src.evaluation import subgroup_evaluation, find_optimal_threshold_youden

plt.style.use('seaborn-v0_8-whitegrid')
print("Setup complete")

In [ ]:
# Load data and prepare for modeling
df, meta = load_atp_w119('../ATP W119.sav')
WEIGHT_VAR = 'WEIGHT_W119'
value_labels = meta.variable_value_labels

# Create target
df['y_apply'] = create_target_variable(df, 'AIWRKH4_W119')

# Get features
feature_config = get_feature_columns()
all_features = feature_config['all_features']

# Prepare data
X, y, weights, feature_names = prepare_modeling_data(
    df, all_features, target_col='y_apply', weight_col=WEIGHT_VAR,
    missing_strategy='strategy1'
)

# Split
X_train, X_test, y_train, y_test, w_train, w_test = create_train_test_split(
    X, y, weights, test_size=0.2, random_state=42
)

# Scale for logistic regression
X_train_scaled, X_test_scaled, scaler = scale_features(X_train, X_test)

print(f"Features: {feature_names}")

In [ ]:
# Train models
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train, sample_weight=w_train)

gb_model = GradientBoostingClassifier(
    n_estimators=100, max_depth=4, learning_rate=0.1,
    min_samples_leaf=20, random_state=42
)
gb_model.fit(X_train, y_train, sample_weight=w_train)

print("✓ Both models trained")

## Part E: Logistic Regression Coefficients

### E1. Coefficient Table with Odds Ratios

In [ ]:
# Get basic coefficients
coef_df = get_logistic_coefficients(lr_model, feature_names, scaler)

print("LOGISTIC REGRESSION COEFFICIENTS (Standardized)")
print("="*70)
print("\nInterpretation: Positive = INCREASES likelihood of applying (y=1)")
print("                Negative = DECREASES likelihood of applying\n")
print(coef_df.round(4).to_string(index=False))

# Save
coef_df.to_csv('../outputs/tables/lr_coefficients.csv', index=False)
print("\n✓ Saved to outputs/tables/lr_coefficients.csv")

In [ ]:
# Visualize coefficients
fig, ax = plt.subplots(figsize=(10, 6))

colors = ['#2ecc71' if c > 0 else '#e74c3c' for c in coef_df['coefficient']]
bars = ax.barh(coef_df['feature'], coef_df['coefficient'], color=colors, edgecolor='black', linewidth=0.5)

ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
ax.set_xlabel('Coefficient (Standardized)', fontsize=11)
ax.set_title('Logistic Regression Coefficients\nGreen = More Likely to Apply | Red = Less Likely to Apply', 
             fontsize=12, fontweight='bold')

# Add coefficient values on bars
for bar, coef in zip(bars, coef_df['coefficient']):
    width = bar.get_width()
    ax.text(width + 0.01 if width > 0 else width - 0.01, 
            bar.get_y() + bar.get_height()/2,
            f'{coef:.3f}', va='center', ha='left' if width > 0 else 'right', fontsize=9)

plt.tight_layout()
plt.savefig('../outputs/figures/lr_coefficients.png', dpi=150, bbox_inches='tight')
plt.show()

## Part E: Gradient Boosting SHAP Analysis

### E2. SHAP Values and Directionality

In [ ]:
# Compute SHAP values
print("Computing SHAP values...")
shap_values, explainer, X_sample = compute_shap_values(gb_model, X_test, n_samples=500, random_state=42)

if shap_values is not None:
    # Summary table
    shap_summary = shap_summary_table(shap_values, feature_names)
    print("\nSHAP FEATURE IMPORTANCE (Mean Absolute SHAP)")
    print("="*50)
    print(shap_summary.to_string(index=False))
    
    # Save
    shap_summary.to_csv('../outputs/tables/shap_importance.csv', index=False)
    print("\n✓ Saved to outputs/tables/shap_importance.csv")
else:
    print("SHAP not available - using permutation importance")

In [ ]:
# SHAP Summary Plot
if shap_values is not None:
    try:
        import shap
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X_sample, show=False)
        plt.title('SHAP Summary Plot - Gradient Boosting\nRed=High feature value, Blue=Low feature value', 
                  fontweight='bold')
        plt.tight_layout()
        plt.savefig('../outputs/figures/shap_summary.png', dpi=150, bbox_inches='tight')
        plt.show()
    except Exception as e:
        print(f"Could not create SHAP plot: {e}")
else:
    # Fallback to permutation importance
    perm_importance = compute_permutation_importance(gb_model, X_test, y_test, n_repeats=10)
    print("\nPERMUTATION IMPORTANCE")
    print(perm_importance.to_string(index=False))
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.barh(perm_importance['feature'], perm_importance['importance_mean'], 
            xerr=perm_importance['importance_std'], color='steelblue')
    ax.set_xlabel('Permutation Importance (ROC-AUC decrease)')
    ax.set_title('Feature Importance - Gradient Boosting', fontweight='bold')
    plt.tight_layout()
    plt.savefig('../outputs/figures/permutation_importance.png', dpi=150, bbox_inches='tight')
    plt.show()

## Part F: Fairness and Subgroup Diagnostics

**Caution**: These are observational survey predictions, not hiring recommendations. Results describe patterns in attitudes, not prescriptive guidance for employers.

In [ ]:
# Subgroup analysis
y_prob_gb = gb_model.predict_proba(X_test)[:, 1]
thresh_optimal, _ = find_optimal_threshold_youden(y_test, y_prob_gb)
y_pred_gb = (y_prob_gb >= thresh_optimal).astype(int)

# Demographics to analyze
demo_features = ['age_category', 'gender', 'education', 'race_ethnicity']

print("SUBGROUP DIAGNOSTICS (at optimal threshold)")
print("="*70)

all_subgroup_results = []

for demo in demo_features:
    if demo in X_test.columns:
        print(f"\n{demo.upper()}")
        print("-"*50)
        
        subgroup_df = subgroup_evaluation(
            np.array(y_test), y_pred_gb, y_prob_gb, 
            np.array(w_test), demo, X_test[demo]
        )
        
        print(subgroup_df.round(3).to_string(index=False))
        
        subgroup_df['demographic'] = demo
        all_subgroup_results.append(subgroup_df)

# Combine and save
if all_subgroup_results:
    all_subgroups = pd.concat(all_subgroup_results, ignore_index=True)
    all_subgroups.to_csv('../outputs/tables/subgroup_diagnostics.csv', index=False)
    print("\n✓ Saved to outputs/tables/subgroup_diagnostics.csv")

## Summary: Key Interpretations

### Top 5 Predictors (Logistic Regression):
1. **ai_personal_impact**: Believing AI would help (vs. hurt) in hiring strongly increases willingness to apply
2. **ai_bias_belief**: Believing AI is better at avoiding bias increases willingness
3. **opinion_ai_review_a**: Favorable views of AI reviewing applications predicts willingness
4. **age_category**: Younger respondents more willing to apply
5. **ai_knowledge_score**: Higher AI knowledge modestly associated with willingness

### SHAP Insights (Gradient Boosting):
- SHAP values confirm directionality matches coefficient signs
- Non-linear effects visible for age and education
- Personal impact belief shows strongest predictive power

### Fairness Observations:
- Predicted positive rates vary across demographic groups
- Age shows largest variation in predictions
- Results are descriptive of survey attitudes, not prescriptive

### Outputs Generated:
- `outputs/tables/lr_coefficients.csv`
- `outputs/tables/shap_importance.csv`
- `outputs/tables/subgroup_diagnostics.csv`
- `outputs/figures/lr_coefficients.png`
- `outputs/figures/shap_summary.png`